In [2]:
import nltk
from nltk.parse import pchart
grammar = nltk.PCFG.fromstring("""
    S -> NP VP [1.0]
    PP -> P NP [1.0]
    VP -> V NP [0.7] 
    VP -> VP PP [0.3]
    P -> 'with' [1.0]
    V -> 'saw' [1.0]
    NP -> NP PP [0.4]
    NP -> 'astronomers' [0.1]
    NP -> 'ears' [0.18]
    NP -> 'saw' [0.04]
    NP -> 'stars' [0.18]
    NP -> 'telescopes' [0.1]

    """)

# Convert the NLTK PCFG grammar to a Python dictionary
pcfg_dict = {}
for production in grammar.productions():
    lhs = str(production.lhs())
    rhs = [str(sym) for sym in production.rhs()]
    prob = production.prob()
    
    if lhs in pcfg_dict:
        pcfg_dict[lhs].append((rhs, prob))
    else:
        pcfg_dict[lhs] = [(rhs, prob)]

print(pcfg_dict)
sentence = "astronomers saw stars with ears".split(" ")

parser = pchart.InsideChartParser(grammar)
for t in parser.parse(sentence):
    print(t)
    t.pretty_print(unicodelines=True, nodedist=3*2)
    
# viterbi_parser = nltk.ViterbiParser(grammar)

# for tree in viterbi_parser.parse(sentence):
#     print(tree)

{'S': [(['NP', 'VP'], 1.0)], 'PP': [(['P', 'NP'], 1.0)], 'VP': [(['V', 'NP'], 0.7), (['VP', 'PP'], 0.3)], 'P': [(['with'], 1.0)], 'V': [(['saw'], 1.0)], 'NP': [(['NP', 'PP'], 0.4), (['astronomers'], 0.1), (['ears'], 0.18), (['saw'], 0.04), (['stars'], 0.18), (['telescopes'], 0.1)]}
(S
  (NP astronomers)
  (VP (V saw) (NP (NP stars) (PP (P with) (NP ears))))) (p=0.0009072)
                            S                                        
     ┌──────────────────────┴─────────┐                                    
     │                                VP                             
     │            ┌───────────────────┴────────┐                           
     │            │                            NP                    
     │            │         ┌──────────────────┴─────────┐                 
     │            │         │                            PP          
     │            │         │                  ┌─────────┴────────┐        
     NP           V         NP           